In [1]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
max_degree = 20 # 多项式的最大阶数
n_train, n_test = 100, 100 #训练集和验证集数量都是100个
true_w = np.zeros(max_degree) #分配大量的空间
true_w[0:4] = np.array([5,1.2,-3.4, 5.6])
features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)

In [3]:
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
'''这里数组features每个元素都会分别求0-19次方，单个元素求次方形成一个数组，
再把数组组合起来形成poly_features'''
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)  # `gamma(n)` = (n-1)!
# `labels`的维度: (`n_train` + `n_test`,)
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)


In [4]:
true_w, features, poly_features, labels 
= [torch.tensor(x, dtype=d2l.float32) 
   for x in [true_w, features, poly_features, labels]]

features[:2], poly_features[:2, :], labels[:2]

(tensor([[-0.3762],
         [-0.0535]]),
 tensor([[ 1.0000e+00, -3.7622e-01,  7.0771e-02, -8.8752e-03,  8.3476e-04,
          -6.2811e-05,  3.9385e-06, -2.1168e-07,  9.9546e-09, -4.1613e-10,
           1.5656e-11, -5.3545e-13,  1.6787e-14, -4.8583e-16,  1.3056e-17,
          -3.2745e-19,  7.6997e-21, -1.7040e-22,  3.5615e-24, -7.0522e-26],
         [ 1.0000e+00, -5.3478e-02,  1.4299e-03, -2.5490e-05,  3.4078e-07,
          -3.6448e-09,  3.2486e-11, -2.4818e-13,  1.6590e-15, -9.8579e-18,
           5.2718e-20, -2.5629e-22,  1.1422e-24, -4.6985e-27,  1.7947e-29,
          -6.3986e-32,  2.1386e-34, -6.7276e-37,  1.9987e-39, -5.6262e-42]]),
 tensor([4.1678, 4.8556]))

In [5]:
def evaluate_loss(net, data_iter, loss):
    '''评估给数据集上模型的损失'''
    metric = d2l.Accumulator(2)  #损失总和和损失样本量
    for X, y in data_iter:
        out = net(X)
        y=y.reshape(out.shape)
        l=loss(out,y)
        metric.add(l.sum(), l.numel())
    return metric[0] / metric[1]

In [6]:
def train(train_features, test_features, train_labels, test_labels, 
          num_epochs=400):
    loss = nn.MSELoss()
    input_shape = train_features.shape[-1]
    #不设置偏置，因为我们已经在多项式特种中实现了它
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array((train_features, train_labels.reshape(1,-1)),
                                batch_size)
    test_iter = d2l.load_array((test_features, test_labels.reshape(1,-1)), 
                               batch_size, is_train=False)
    trainer = torch.optim.SGD(net.parameters(), lr=0.01)
    animator = d2l.Accumulator(xlabel = 'epoch', ylabel = 'loss', 
                               yscale='log', xlim=[1, num_epochs], 
                               ylim=[1e-3, 1e2]
                              ,legend=['train', 'test'])
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (rpoch+1) % 20==0:
            animator.add(epoch+1, (evaluate_loss(net, trian_iter, loss),
                                  evaluate_loss(net, test_iter, loss)))
    print('weight:', net[0].weight.data.numpy())

In [7]:
# 从多项式特征中选择前4个维度，即 1, x, x^2/2!, x^3/3!
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
      labels[:n_train], labels[n_train:])
# 从多项式特征中选择前2个维度，即 1, x
train(poly_features[:n_train, :2], poly_features[n_train:, :2],
      labels[:n_train], labels[n_train:])
# 从多项式特征中选取所有维度
train(poly_features[:n_train, :], poly_features[n_train:, :],
      labels[:n_train], labels[n_train:], num_epochs=1500)

AssertionError: Size mismatch between tensors